In [8]:
import os
import pandas as pd
import glob
import stagger
import librosa
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
from librosa import display
from librosa import feature
from stagger import read_tag


In [9]:
#wykrycie ścieżki, w której przechowuje pliki i program
working_directory = os.getcwd()
path = working_directory + '/wrapped/big data projekt/*.mp3' 
songs = glob.glob(path) 


In [10]:
"osobna funkcja do liczenia sredniej z macierzy"
def Average(lst):
    return sum(lst) / len(lst)


In [11]:

"""stworzenie pustych list na 20 wskaznikow mfcc"""
mfcc_0, mfcc_1, mfcc_2, mfcc_3, mfcc_4, mfcc_5, mfcc_6, mfcc_7, mfcc_8, mfcc_9, mfcc_10, mfcc_11, mfcc_12, mfcc_13, mfcc_14, mfcc_15, mfcc_16, mfcc_17, mfcc_18, mfcc_19 = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
mfcc_lists = [mfcc_0 , mfcc_1 ,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19]


In [12]:

"""stworzenie listy cech jednowymiarowych"""
feature1_list_i = [
 feature.chroma_stft,
 feature.spectral_centroid,
 feature.spectral_bandwidth,
 feature.spectral_rolloff ]
 
"""stworzenie listy cech dwuwymiarowych"""
feature2_list_ii = [
 feature.rms,
 feature.zero_crossing_rate ]

"""stworzenie listy cech mfcc"""
feature3_list_mfcc = [
    feature.mfcc ]


In [13]:


"""stworzenie funkcji obliczającej średnią z danej cechy i zwracającej wyniki do wektora feature_vector"""
def get_feature_vector(y,sr,mp3): 
    feat_artist = [mp3.artist]
    feat_vect_i = [ np.mean(funct(y=y,sr=sr)) for funct in feature1_list_i]
    feat_vect_ii = [ np.mean(funct(y=y)) for funct in feature2_list_ii] 
    mfcc = [np.mean(funct(y=y,sr=sr).T, axis = 0) for funct in feature3_list_mfcc]
    feat_vect_iii = list(mfcc[0])
    
    #feature_vector jako lista przechowująca wszystkie cechy
    feature_vector = feat_artist + feat_vect_i + feat_vect_ii + feat_vect_iii
    return feature_vector


In [57]:
audio_features = []
licznik = 0
for song in songs:
    mp3 = stagger.read_tag(song)
    print(f'dodaje piosenke: {mp3.title}')
    dl = librosa.get_duration(path=song)
    #odebranie sygnalu dla calej piosenki
    y , sr = librosa.load(song,sr = 22858)
    
    #wyelminowanie tła i zostawienie wokalu
    S_full, phase = librosa.magphase(librosa.stft(y=y))
    S_filter = librosa.decompose.nn_filter(S_full,
                                       aggregate=np.median,
                                       metric='cosine',
                                       width=int(librosa.time_to_frames(2, sr=sr)))
    #stworzenie filtra
    S_filter = np.minimum(S_full, S_filter)
    margin_i, margin_v = 2, 10
    power = 2

    mask_i = librosa.util.softmask(S_filter,
                               margin_i * (S_full - S_filter),
                               power=power)

    mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)
    S_foreground = mask_v * S_full
    S_background = mask_i * S_full
    y_foreground = librosa.istft(S_foreground * phase)

    S_foreground = mask_v * S_full
    S_background = mask_i * S_full
    
    feature_vector = get_feature_vector(y_foreground, sr,mp3)
    audio_features.append(feature_vector)
    licznik+=1
    

licznik: 0
dodaje piosenke: (Nice Dream)
licznik: 1
dodaje piosenke: 10:15 Saturday Night
licznik: 2
dodaje piosenke: 15 Step
licznik: 3
dodaje piosenke: 2 + 2 = 5
licznik: 4
dodaje piosenke: A Forest
licznik: 5
dodaje piosenke: A Punch Up at a Wedding
licznik: 6
dodaje piosenke: A Short Term Effect
licznik: 7
dodaje piosenke: A Wolf At the Door
licznik: 8
dodaje piosenke: Airbag
licznik: 9
dodaje piosenke: All I Need
licznik: 10
dodaje piosenke: Always in the Sun
licznik: 11
dodaje piosenke: Amour dans le motu
licznik: 12
dodaje piosenke: Antitaxi
licznik: 13
dodaje piosenke: Anyone Can Play Guitar
licznik: 14
dodaje piosenke: Ashes to Ashes - 2017 Remaster
licznik: 15
dodaje piosenke: Backdrifts
licznik: 16
dodaje piosenke: bitter cold
licznik: 17
dodaje piosenke: Black Star
licznik: 18
dodaje piosenke: Blackout - 2017 Remaster
licznik: 19
dodaje piosenke: Blackstar
licznik: 20
dodaje piosenke: Bloom
licznik: 21
dodaje piosenke: Blow Out
licznik: 22
dodaje piosenke: Bodysnatchers
lic

In [ ]:
df = pd.DataFrame(audio_features,columns = header)

In [ ]:
writer = pd.ExcelWriter('wrapped_dataset_vocal_270.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='welcome', index=False)
writer.save()